[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/Digital-Finance-Introduction/blob/main/day_04/notebooks/NB09_AMM_Simulation.ipynb)

# NB09: AMM & Impermanent Loss

**Topic:** 4.2 - Automated Market Makers

**Learning Objectives:**
- Understand x*y=k formula
- Calculate impermanent loss
- Simulate AMM swaps

## 1. Setup

In [ ]:
!pip install -q numpy pandas plotly
import numpy as np
import pandas as pd
print("Ready")